# Korean word embedding

한국어 데이터셋을 읽고 word score 계산, tokenizing, word2vec 모델을 학습시키고 단어에 대한 벡터를 반환하는 클래스를 구현합니다.

## 준비
아래의 코드를 돌리기 위해서는 3가지의 pip install이 필요합니다

    pip install soynlp
    pip install gensim
    pip install numpy==1.13, should downgrade numpy

In [1]:
import pandas as pd

import re

import torch

from soynlp.word import WordExtractor
from soynlp.tokenizer import LTokenizer, MaxScoreTokenizer

import warnings
warnings.filterwarnings(action='ignore', category=UserWarning, module='gensim')

from gensim.models import Word2Vec

import pickle

# pip install soynlp
# pip install gensim
# pip install numpy==1.13, should downgrade numpy

# https://github.com/lovit/soynlp/
# https://lovit.github.io/nlp/2018/04/09/three_tokenizers_soynlp/
# https://ratsgo.github.io/natural%20language%20processing/2017/03/08/word2vec/
# https://radimrehurek.com/gensim/models/word2vec.html
# https://rutumulkar.com/blog/2015/word2vec

# 띄어쓰기 오류의 해결 (맞춤법 교정 툴 사용)
# 학습되지 않은 단어에 대한 vectorization

## Embedding

Embedding 내부의 tokenizer, word2vec 모델을 훈련시키기 위해서는 setWord2Vec 함수의 호출이 필요합니다.
이때, 함수의 매개변수로 데이터셋의 파일 이름이 필요하며, 기본적으로 .xlsx 파일을 사용하도록 되어있습니다.

함수 실행 시, 읽어온 데이터셋을 바탕으로 word score 계산 -> tokenizer 훈련 -> word2vec 모델 훈련이 이루어지게 됩니다

In [2]:
class Embedding:
    def setWord2Vec(self, fileName):
        question = pd.read_excel(fileName + '.xlsx')['question']
        print(' read question data from ', fileName)        
        for i in range(0,len(question)):
            question[i] = self.onlyKorean(question[i])
            
        word_scores = self.calWordScores(question)
        self.tokenizer = self.trainTokenizer(word_scores)        
        self.word2vec = self.trainWord2Vec(question)
        
    def calWordScores(self, question):   
        word_extractor = WordExtractor(
            max_left_length=20, 
            max_right_length=20, 
            min_frequency = 20,
            min_cohesion_forward = 0.05,
            min_right_branching_entropy = 0.0
        )
        
        word_extractor.train(question)   
        word_scores = word_extractor.extract()
        print(' extract and calculate ', len(word_scores), ' words')
        return word_scores
    
    def onlyKorean(self, sentence):
        korean = re.compile('[^ ㄱ-ㅣ가-힣]+') 
        result = korean.sub('', sentence)
        return result
    
    def trainTokenizer(self, word_scores):
        cohesion_scores = {word:score.cohesion_forward for word, score in word_scores.items()}
        tokenizer = MaxScoreTokenizer(scores = cohesion_scores)
        # tokenizer = LTokenizer(scores = cohesion_scores)
        print(' train tokenizer')  
        return tokenizer
                
    def trainWord2Vec(self, question):
        # print(self.question)
        tQuestion = [self.tokenizeSentence(q) for q in question]
        
        word2vec = Word2Vec(
            tQuestion, 
            size = 50, 
            window = 2, 
            min_count = 1, 
            iter = 100, 
            sg = 1
        )
        print(' train word2vec') 
        return word2vec
    
    # sent는 하나의 문장
    def tokenizeSentence(self, sent): 
        return self.tokenizer.tokenize(sent)
    
    # words 는 단어들의 리스트 ["김동호", "교수님"]
    def vectorizeWord(self, words):          
        return self.word2vec.wv[words] 
    
    def vectorizeSentence(self, sent):
        result = []
        for s in sent:
            s = self.onlyKorean(s)
            tSent = self.tokenizeSentence(s) 
            vec = self.vectorizeWord(tSent)
            result.append(torch.FloatTensor(vec))
        return result

## Embedding 객체 생성

현재, '통합_181204.xlsx' 데이터셋을 사용하도록 되어있습니다. 다른 데이터셋을 사용하고자 하실 경우 아래의 셀에서 파일명을 수정하여 주시기 바랍니다. 데이터셋과 본 코드는 같은 디렉토리에 있어야 합니다.

In [3]:
embed = Embedding()

embed.setWord2Vec("통합_181204")

 read question data from  통합_181204
training was done. used memory 0.149 Gbry 0.149 Gb
all cohesion probabilities was computed. # words = 193
all branching entropies was computed # words = 2017
all accessor variety was computed # words = 2017
 extract and calculate  232  words
 train tokenizer
 train word2vec


In [4]:
with open('Embedding.pkl', 'wb') as f:
    pickle.dump(embed, f)